In [1]:
# STEP 0: QEPC project bootstrap (works from any notebook folder)

import sys
from pathlib import Path

def find_project_root() -> Path:
    """
    Walk upwards from the current working directory until we find
    a folder that looks like the QEPC project root
    (has main.py and a qepc/ package).
    """
    here = Path.cwd()
    for parent in [here] + list(here.parents):
        if (parent / "main.py").exists() and (parent / "qepc").exists():
            return parent
    # Fallback: just use cwd
    return here

PROJECT_ROOT = find_project_root()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("QEPC project root:", PROJECT_ROOT)

from qepc_autoload import qepc_step
import pandas as pd

qepc_step("QEPC smoketest notebook started")


QEPC project root: C:\Users\wdorsey\qepc_project
[QEPC Paths] Project Root set: C:\Users\wdorsey\qepc_project
[QEPC] Autoload complete.


In [2]:
# STEP 1: Import strengths, lambda engine, simulator

from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths
from qepc.core.lambda_engine import compute_lambda
from qepc.core.simulator import run_qepc_simulation

qepc_step("Imported strengths_v2, lambda_engine, and simulator successfully")


In [3]:
# STEP 2: Calculate team strengths from Team_Stats

strengths_df = calculate_advanced_strengths(
    cutoff_date=None,   # or "2025-11-01" etc if you want a backtest-style cutoff
    verbose=True,
)

display(strengths_df.head())
print(f"Total teams in strengths_df: {len(strengths_df)}")


AttributeError: module 'numpy' has no attribute 'read_csv'

In [ ]:
# STEP 3: Create a tiny schedule with 1 test game

qepc_step("Building a tiny test schedule")

# Pick first two teams
home_team = strengths_df.iloc[0]["Team"]
away_team = strengths_df.iloc[1]["Team"]

print("Home team:", home_team)
print("Away team:", away_team)

schedule_df = pd.DataFrame(
    {
        "Home Team": [home_team],
        "Away Team": [away_team],
        # Optional situational columns (we'll leave them out for now):
        # "rest_days_home": [2],
        # "rest_days_away": [1],
        # "home_b2b": [False],
        # "away_b2b": [False],
    }
)

display(schedule_df)


In [ ]:
# STEP 4: Compute lambda for the test game

qepc_step("Computing lambda for test game")

lambda_df = compute_lambda(
    schedule_df=schedule_df,
    team_stats_df=strengths_df,
    include_situational=False,  # True if you add rest_days_* & *_b2b columns
)

display(lambda_df[["Home Team", "Away Team", "lambda_home", "lambda_away", "vol_home", "vol_away"]])


In [ ]:
# STEP 5: Simulate the game

qepc_step("Running QEPC simulation")

sim_results = run_qepc_simulation(
    df=lambda_df,
    num_trials=5000,      # enough for a stable feel but not too slow
    use_normal=True,      # NBA-friendly
    correlated_scores=True,
)

display(
    sim_results[
        [
            "Home Team",
            "Away Team",
            "lambda_home",
            "lambda_away",
            "Sim_Home_Score",
            "Sim_Away_Score",
            "Home_Win_Prob",
            "Away_Win_Prob",
            "Expected_Spread",
            "Expected_Score_Total",
        ]
    ]
)
